In [27]:
import re
import json
import random
#infile = 'papersub.txt'
#outfile = 'papersubq-word.tsv'
infile = 'problemsheet2text.json'
outfile = 'a2.jsonl'

acccnt = 0
errcnt = 0

with open(infile, 'r') as f:
    json_data = json.load(f)

out=open(outfile,"w");

regex = r"[0-9]*\.?[0-9]+\.?[0-9]*%?"
#regex = r"[0-9]*\.?[0-9]+\.?[0-9]*"
r = re.compile(regex)
#m = r.search(t);

jsonls = []
i= 0
#for line in lines:
for d in json_data:
    i = i+1;
    if(i%1000==0):
        print(str(i))
    '''
    if(i>1000):
        break;
    '''   
    '''
    t = line.split("\t");
    s = t[0];
    reg = t[1].strip();
    '''
    s=d['para']
    #reg=d['pat']
    problem_no = d['problem_no']
    task_no = d['task_no']
    doc_file = d['doc_file']
    question = d['question']
    answer_form = d['answer_form']
    #sent = d['sent']
    sent = d['para']
    
    jsonl = d;
    #jsonl['task_no']=d['task_no']
    jsonl['text'] = sent
    jsonl['team_id']=""
    jsonl['hash']=""
    ####jsonl['answer']=jsonl['answer_form']
    label = []
    answer_auto = []
    
    
    
    #print(str(s.find(',')));
    if(s.find(',')<0):
        continue;
    #문장 초반에(5) 콤마 나오면 삭제
    if(s.find(',')<5):
        s = s[s.find(',')+1:]
    s = s.strip();
    
    
    coms = s.split(',');
    ccnt = len(coms);
    pre = coms[0];
    
    twlen = 0
    ncnt = 0
    #print(s);
    for com in coms[1:-1]:
        #m = r.search(com)
        #m = re.match(regex, com)
        m = re.findall(regex,com)
        
        if(m != None):
            if(len(m)>0):
                #print(str(m.groups()))
                #####print(str(m))
                ncnt = ncnt+1
        
        #print(com)
        ws = com.split(' ');
        wcnt = len(ws);
        twlen = twlen+wcnt
    cccnt = ccnt - 2
    avgwlen = int(twlen/cccnt) -1
    #if(avgwlen<=3):
    #if(avgwlen<=2 and avgwlen>=2):
    #####if(avgwlen<=4 and ncnt > 0):
    if(avgwlen<=3 and ncnt == 0):
        ####print("\n"+str(avgwlen) + " " + s+"\n")
        pass;
    else:
        continue;
    
    
    #avgwlen 기준으로 접두 생성중인데, 접두에 숫자 있는경우엔 숫자기준으로 변경 필요해보임
    #접미에도 숫자있는경우엔 숫자기준으로 변경 필요해보임
    pret = coms[0].split(' ');
    pre = ' '.join(pret[0:-1*avgwlen]);
    
    
    sut = coms[-1].split(' ');
    su = ' '.join(sut[avgwlen+1:]);
    #### 접두, 접미, coms 생성완료
    #print("pre: "+ pre);
    #print("su: "+su+"\n");
    
    j = 0
    for com in coms:
        j = j+1
        ansnum = com
        if(j == 1): 
            ansnum = ' '.join(pret[-1*avgwlen:]);
        if(j==len(coms)):
            ansnum = ' '.join(sut[:avgwlen+1]);
        #print("com: "+com);
        #print("ansnum: "+ansnum);
        
        if(com==""):
            continue;
        start = sent.find(ansnum);
        end = sent.find(ansnum)+len(ansnum)
        l=[]
        l.append(start)
        l.append(end)
        l.append("answer");
        
        answer_auto.append(ansnum);
        label.append(l)
        
    '''    
    answers_unique = list(set(answer_auto))
    if(len(answers_unique)!=len(answer_auto)):
        continue;
    '''
    
    #len(answer_auto)이 answer_form보다 1 더 큰 경우 맨 앞 정답후보를 버려서 정렬함
    if(len(answer_auto)-len(answer_form)==1):
        answer_auto = answer_auto[1:]
    if(len(answer_auto)-len(answer_form)==-1):
        answer_auto.append(answer_auto[len(answer_auto)-1])

    
    j = -1
    for form in answer_form:
        j = j+1;
        ####print(str(j))
        ####print(str(answer_auto))
        
        ans = answer_auto[min(j,len(answer_auto)-1)]
        #print(str(len(ans))+"\t"+str((ans.find('('))))
        #print(ans)
        #ans = ans[:max(len(ans),ans.find('('))]
        f = ans.find('(')
        if(f==-1):
            f=len(ans)
        ans = ans[:min(len(ans),f)]
        #print(ans)
        
        
        ans = ans.strip();
        form['predic']=ans
        
        if (form['answer'].find(form['predic'])>=0):
            acccnt = acccnt+1
        else:
            errcnt = errcnt+1
        ####form.append(pred)
    
    jsonl['answer'] = answer_form
    jsonl['answer_auto'] = answer_auto
    jsonl.pop('answer_form')
    #doccano 버전 낮을때는 'labels'
    #jsonl['label'] = label
    ####jsonl['labels'] = label
    
    jsonls.append(jsonl)
    

print(str(acccnt)+"\t"+str(errcnt)) 

#random.shuffle(jsonls)
out.write(json.dumps(jsonls,ensure_ascii=False,indent=4))

'''
jsontxts = []
for jsonl in jsonls:
    #jsontxts.append(json.dumps(jsonl, indent=4, ensure_ascii=False))
    jsontxts.append(json.dumps(jsonl, ensure_ascii=False))
    
c = "\n".join(jsontxts)
out.write(c);
'''        
        
        
      
    
out.close();

265	110
